# Import Libraries and Datasets

In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf

from src.constants import IMAGE_SIZE, CHANNELS, BATCH_SIZE, EPOCHS
from src.preprocessing_functions import create_generator
from src.models import build_cnn_model, check_gpu, train_cnn_model, plot_training_validation_history, evaluate_train_test_performance

In [2]:
# Load paths in a dict for train, test, valid folders with wildfire and nowildfire images 

images_path = '/Users/silvanoquarto/Desktop/PROJECTS/Data_Wildfire_Project'
splits = ['train', 'test', 'valid']
labels = ['wildfire', 'nowildfire']

image_paths = {f"{split}_{label}": [] for split in splits for label in labels}

for split in splits:
    for label in labels:
        path = os.path.join(images_path, split, label)
        if os.path.exists(path):  
            image_files = os.listdir(path)
            full_paths = [os.path.join(path, img) for img in image_files]
            image_paths[f"{split}_{label}"].extend(full_paths)
            print(f"- {split.capitalize()} set for {label} uploaded correctly!!")
        else:
            print(f"Warning: {path} does not exist :(")

- Train set for wildfire uploaded correctly!!
- Train set for nowildfire uploaded correctly!!
- Test set for wildfire uploaded correctly!!
- Test set for nowildfire uploaded correctly!!
- Valid set for wildfire uploaded correctly!!
- Valid set for nowildfire uploaded correctly!!


# IMAGE PREPROCESSING

In [3]:
# Data augumentation and normalization
 
print('Data augumentation and normalization for training...')
train_generator = create_generator(images_path, 'train')

print('Data augumentation and normalization for validation...')
validation_generator = create_generator(images_path, 'valid')

print('Data augumentation and normalization for test...')
test_generator = create_generator(images_path, 'test')

Data augumentation and normalization for training...
Found 30249 images belonging to 2 classes.
Data augumentation and normalization for validation...
Found 6300 images belonging to 2 classes.
Data augumentation and normalization for test...
Found 6299 images belonging to 2 classes.


# Build and Train CNN model from scratch

## Build model

In [4]:
model = build_cnn_model(input_shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS), num_output=1, learning_rate=0.001, dropout_rate=0.5)

model.summary()

2024-06-24 17:08:26.737376: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2024-06-24 17:08:26.737400: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-06-24 17:08:26.737403: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-06-24 17:08:26.737620: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-24 17:08:26.737982: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 348, 348, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 174, 174, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 172, 172, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 86, 86, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 84, 84, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 42, 42, 128)       0

## Train model

In [5]:
# Check if GPU is available

gpu_available = check_gpu()
print("GPU available:", gpu_available)


All Physical Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU details: {'device_name': 'METAL'}
GPU available: True


In [7]:
history = train_cnn_model(
    model,
    train_generator,
    validation_generator,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1
)

Epoch 1/5


2024-06-24 17:08:47.231626: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2024-06-24 17:08:47.287568: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


ResourceExhaustedError: Graph execution error:

Detected at node sequential/conv2d/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/Users/silvanoquarto/Desktop/PROJECTS/Wildfire_Prediction_with_Satellite_Images/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/Users/silvanoquarto/Desktop/PROJECTS/Wildfire_Prediction_with_Satellite_Images/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/Users/silvanoquarto/Desktop/PROJECTS/Wildfire_Prediction_with_Satellite_Images/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/Users/silvanoquarto/Desktop/PROJECTS/Wildfire_Prediction_with_Satellite_Images/.venv/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/opt/homebrew/Cellar/python@3.11/3.11.9/Frameworks/Python.frame

# Model evaluation

In [ ]:
# Training vs Validation performances

plot_training_validation_history(history)

In [ ]:
# Training vs Test performance

evaluate_train_test_performance(model, train_generator, test_generator)

# Save model

In [ ]:
# Save the model

model.save('wildfire_detection_cnn.h5')